In [1]:
import zipfile
from zipfile import ZipFile
import pandas as pd
import numpy as np
import netCDF4 as nc
from matplotlib import pyplot as plt
import xarray as xr
import os
from calendar import isleap
import warnings
warnings.filterwarnings("ignore")
import copy
import xarray as xr
from rasterio import Affine
from rasterstats import zonal_stats
from osgeo import gdal
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon

In [2]:
from shapely.ops import unary_union
gdf = gpd.read_file(r"Path\To\File\GTA.shp")
merged = gdf.geometry.unary_union
gdf_merged = gpd.GeoDataFrame(geometry=[merged])
gdf_merged = gdf_merged.set_crs('EPSG:4326')
gdf_m = gdf_merged.to_crs('EPSG:3857')
gdf_m['area_m2'] = gdf_m.geometry.area
areaofshp=gdf_m['area_m2']

In [1]:
# Future Surface Temp 585
path="Path\To\File\surface temperature\ssp585 CMCC-CM2-SR5 r1i1p1f1"
basedir="Path\To\File\surface temperature\ssp585 CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'tslsi'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Surface_Temp_Daily_Future_585=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Surface_Temp_Daily_Future_585['Future Surface Temp']=Data
Surface_Temp_Daily_Future_585['Date']=Date
Surface_Temp_Daily_Future_585['Future Surface Temp']=Surface_Temp_Daily_Future_585['Future Surface Temp']-273.15

NameError: name 'gdf_merged' is not defined

In [4]:
Surface_Temp_Daily_Future_585

,Future Surface Temp,Date
0,-0.764327,2015-01-01 12:00:00
1,0.302564,2015-01-02 12:00:00
2,-0.748459,2015-01-03 12:00:00
3,1.362586,2015-01-04 12:00:00
4,2.014114,2015-01-05 12:00:00
...,...,...
31385,6.705941,2100-12-27 12:00:00
31386,6.312170,2100-12-28 12:00:00
31387,4.447616,2100-12-29 12:00:00
31388,4.466466,2100-12-30 12:00:00


In [5]:
# Future Surface Temp 245
path="Path\To\File\surface temperature\ssp245 CMCC-CM2-SR5 r1i1p1f1"
basedir="Path\To\File\surface temperature\ssp245 CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'tslsi'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area

            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Surface_Temp_Daily_Future_245=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Surface_Temp_Daily_Future_245['Future Surface Temp']=Data
Surface_Temp_Daily_Future_245['Date']=Date
Surface_Temp_Daily_Future_245['Future Surface Temp']=Surface_Temp_Daily_Future_245['Future Surface Temp']-273.15

In [6]:
Surface_Temp_Daily_Future_245

,Future Surface Temp,Date
0,-0.791463,2015-01-01 12:00:00
1,0.214189,2015-01-02 12:00:00
2,-1.584178,2015-01-03 12:00:00
3,1.290696,2015-01-04 12:00:00
4,1.833790,2015-01-05 12:00:00
...,...,...
31385,4.816925,2100-12-27 12:00:00
31386,4.777284,2100-12-28 12:00:00
31387,4.138953,2100-12-29 12:00:00
31388,6.578263,2100-12-30 12:00:00


In [7]:
# Historical Surface Temp
path="Path\To\File\surface temperature\Hist CMCC-CM2-SR5 r1i1p1f1"
basedir="Path\To\File\surface temperature\Hist CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'tslsi'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area

            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Surface_Temp_Daily_Historical=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Surface_Temp_Daily_Historical['Historical Surface Temp']=Data
Surface_Temp_Daily_Historical['Date']=Date
Surface_Temp_Daily_Historical['Historical Surface Temp']=Surface_Temp_Daily_Historical['Historical Surface Temp']-273.15

In [8]:
Surface_Temp_Daily_Historical

,Historical Surface Temp,Date
0,-3.206376,1950-01-01 12:00:00
1,-4.793683,1950-01-02 12:00:00
2,-6.342786,1950-01-03 12:00:00
3,-0.614595,1950-01-04 12:00:00
4,1.133996,1950-01-05 12:00:00
...,...,...
23720,0.176653,2014-12-27 12:00:00
23721,1.311128,2014-12-28 12:00:00
23722,-1.060697,2014-12-29 12:00:00
23723,-4.425489,2014-12-30 12:00:00


In [9]:
# Future LAI 245
path="Path\To\File\LAI\ssp245 CMCC-CM2-SR5 r1i1p1f1"
basedir="Path\To\File\LAI\ssp245 CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'lai'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


LAI_Daily_Future_245=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
LAI_Daily_Future_245['Future LAI']=Data
LAI_Daily_Future_245['Date']=Date


In [10]:
LAI_Daily_Future_245

,Future LAI,Date
0,2.605149,2015-01-01 12:00:00
1,2.600224,2015-01-02 12:00:00
2,2.594446,2015-01-03 12:00:00
3,2.587816,2015-01-04 12:00:00
4,2.580334,2015-01-05 12:00:00
...,...,...
31385,3.576860,2100-12-27 12:00:00
31386,3.576411,2100-12-28 12:00:00
31387,3.575962,2100-12-29 12:00:00
31388,3.575514,2100-12-30 12:00:00


In [11]:
# Future LAI 585
path="Path\To\File\LAI\ssp585 CMCC-CM2-SR5 r1i1p1f1"
basedir="Path\To\File\LAI\ssp585 CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'lai'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


LAI_Daily_Future_585=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
LAI_Daily_Future_585['Future LAI']=Data
LAI_Daily_Future_585['Date']=Date


In [12]:
LAI_Daily_Future_585

,Future LAI,Date
0,2.605151,2015-01-01 12:00:00
1,2.600231,2015-01-02 12:00:00
2,2.594458,2015-01-03 12:00:00
3,2.587833,2015-01-04 12:00:00
4,2.580356,2015-01-05 12:00:00
...,...,...
31385,2.962144,2100-12-27 12:00:00
31386,2.961739,2100-12-28 12:00:00
31387,2.961335,2100-12-29 12:00:00
31388,2.960930,2100-12-30 12:00:00


In [13]:
# Historical LAI
path="Path\To\File\LAI\Hist CMCC-CM2-SR5 r1i1p1f1"
basedir="Path\To\File\LAI\Hist CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'lai'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


LAI_Daily_Historical=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
LAI_Daily_Historical['Historical LAI']=Data
LAI_Daily_Historical['Date']=Date


In [14]:
LAI_Daily_Historical

,Historical LAI,Date
0,2.356795,1950-01-01 12:00:00
1,2.263609,1950-01-02 12:00:00
2,2.213665,1950-01-03 12:00:00
3,2.147241,1950-01-04 12:00:00
4,2.061850,1950-01-05 12:00:00
...,...,...
23720,2.617087,2014-12-27 12:00:00
23721,2.616372,2014-12-28 12:00:00
23722,2.614845,2014-12-29 12:00:00
23723,2.612464,2014-12-30 12:00:00


In [15]:
# Future Precip 245
path="Path\To\File\Precip\ssp245 CMCC-CM2-SR5 r1i1p1f1"
basedir="Path\To\File\Precip\ssp245 CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'pr'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Precip_Daily_Future_245=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Precip_Daily_Future_245['Future Precip']=Data
Precip_Daily_Future_245['Date']=Date
Precip_Daily_Future_245['Future Precip']=Precip_Daily_Future_245['Future Precip']*86400

In [16]:
# Future Precip 585
path="Path\To\File\Precip\ssp585 CMCC-CM2-SR5 r1i1p1f1"
basedir="Path\To\File\Precip\ssp585 CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'pr'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Precip_Daily_Future_585=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Precip_Daily_Future_585['Future Precip']=Data
Precip_Daily_Future_585['Date']=Date
Precip_Daily_Future_585['Future Precip']=Precip_Daily_Future_585['Future Precip']*86400

In [17]:
# Historical Precip
path="Path\To\File\Precip\Hist CMCC-CM2-SR5 r1i1p1f1"
basedir="Path\To\File\Precip\Hist CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'pr'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Precip_Daily_Historical=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Precip_Daily_Historical['Historical Precip']=Data
Precip_Daily_Historical['Date']=Date
Precip_Daily_Historical['Historical Precip']=Precip_Daily_Historical['Historical Precip']*86400

In [18]:
Precip_Daily_Historical

,Historical Precip,Date
0,2.095254,1950-01-01 12:00:00
1,0.293760,1950-01-02 12:00:00
2,0.162074,1950-01-03 12:00:00
3,0.776312,1950-01-04 12:00:00
4,23.249450,1950-01-05 12:00:00
...,...,...
23720,0.266665,2014-12-27 12:00:00
23721,0.024791,2014-12-28 12:00:00
23722,2.020759,2014-12-29 12:00:00
23723,1.863580,2014-12-30 12:00:00


In [19]:
# Future Runoff 245
path="Path\To\File\Runoff\ssp245 CMCC-CM2-SR5 r1i1p1f1"
basedir="Path\To\File\Runoff\ssp245 CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'mrro'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Runoff_Daily_Future_245=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Runoff_Daily_Future_245['Future Runoff']=Data
Runoff_Daily_Future_245['Date']=Date
Runoff_Daily_Future_245['Future Runoff']=Runoff_Daily_Future_245['Future Runoff']*(areaofshp[0])*0.001
Runoff_Daily_Future_245['Future_Runoff_Depth']=Runoff_Daily_Future_245['Future Runoff']*24*60*60/(areaofshp[0])

In [20]:
ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 192, lon: 288, time: 13140)
Coordinates:
  * time       (time) object 2065-01-01 12:00:00 ... 2100-12-31 12:00:00
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 -180.0 -178.8 -177.5 -176.2 ... 176.2 177.5 178.8
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    mrro       (time, lat, lon) float32 ...
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   60225.0
    branch_time_in_parent:  60225.0
    comment:                none
    ...                     ...
    title:                  CMCC-CM2-SR5 output prepared for CMIP6
    variable_id:            mrro
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by CMCC is licensed und...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/8ffe0c9b-33e3-4604-b372-61cf8776158c

In [21]:
# Future Runoff 585
path="Path\To\File\Runoff\ssp585 CMCC-CM2-SR5 r1i1p1f1"
basedir="Path\To\File\Runoff\ssp585 CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'mrro'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.average(mini)


Runoff_Daily_Future_585=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Runoff_Daily_Future_585['Future Runoff']=Data
Runoff_Daily_Future_585['Date']=Date
Runoff_Daily_Future_585['Future Runoff']=Runoff_Daily_Future_585['Future Runoff']*(areaofshp[0])*0.001
Runoff_Daily_Future_585['Future_Runoff_Depth']=Runoff_Daily_Future_585['Future Runoff']*24*60*60/(areaofshp[0])


In [22]:
# Historical Runoff
path="Path\To\File\Runoff\Hist CMCC-CM2-SR5 r1i1p1f1"
basedir="Path\To\File\Runoff\Hist CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'mrro'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


Runoff_Daily_Historical=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
Runoff_Daily_Historical['Historical Runoff']=Data
Runoff_Daily_Historical['Date']=Date
Runoff_Daily_Historical['Historical Runoff']=Runoff_Daily_Historical['Historical Runoff']*(areaofshp[0])*0.001
Runoff_Daily_Historical['Historical_Runoff_Depth']=Runoff_Daily_Historical['Historical Runoff']*24*60*60/(areaofshp[0])

In [23]:
# Future SM 245
path=r"Path\To\File\upper sm content\ssp245 CMCC-CM2-SR5 r1i1p1f1"
basedir=r"Path\To\File\upper sm content\ssp245 CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'mrsos'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


SM_Daily_Future_245=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
SM_Daily_Future_245['Future SM']=Data
SM_Daily_Future_245['Date']=Date
SM_Daily_Future_245['Future SM']=SM_Daily_Future_245['Future SM']/100

In [24]:
# Future SM 585
path=r"Path\To\File\upper sm content\ssp585 CMCC-CM2-SR5 r1i1p1f1"
basedir=r"Path\To\File\upper sm content\ssp585 CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'mrsos'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


SM_Daily_Future_585=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
SM_Daily_Future_585['Future SM']=Data
SM_Daily_Future_585['Date']=Date
SM_Daily_Future_585['Future SM']=SM_Daily_Future_585['Future SM']/100

In [25]:
# Historical Soil Moisture
path=r"Path\To\File\upper sm content\Hist CMCC-CM2-SR5 r1i1p1f1"
basedir=r"Path\To\File\upper sm content\Hist CMCC-CM2-SR5 r1i1p1f1"
path_poly = r"Path\To\File\GTA.shp" 

# print('Polygon coordinates: {}'.format(geom.exterior.coords.xy))
average = {}
filepathname=[]


gdf = gdf_merged
geom = gdf.geometry.values[0]

for file in os.listdir(basedir):
    filepathname.append(os.path.join(basedir, file))

for path_nc in filepathname:
    ds = xr.open_dataset(path_nc)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')
    gdobj = gdal.Open(path_nc)
    
    gt = list(gdobj.GetGeoTransform()) # [-0.625, 1.25, 0.0, 90.47120418848168, 0.0, -0.9424083769633508]
    gt[0] = -180 - (gt[1]/2) # shift the x coordinate by half a pixel
    gt[3] = 90 - (gt[5]/2)
    affine = Affine.from_gdal(*gt)
   
    var = 'mrsos'

    
    for t in ds.time.data:
#         t= ds.time.data[0]
        d = ds.sel(time=t)
        xds = xr.Dataset({var: (["y", "x"], d[var].data[::-1, :])},  # reverse to be consistent with y
                              coords={"x": (["x"], d.lon.data),
                                       "y": (["y"], d.lat.data[::-1])}) # reverse so that Affine is not -ve
        array = xds[var].values

        # zonal stats
        zstats = zonal_stats(geom, array, affine=affine, stats="count", nodata=np.nan, all_touched=True, raster_out=True)
        mini = zstats[0]['mini_raster_array']
        ma = zstats[0]['mini_raster_affine']
        if zstats[0]['count'] < 20:

            # overlay geometry with mini_raster_array to compute the area of the
            # polygon within each pixel

            f_overlap = np.zeros(mini.shape)
            for i in range(0, mini.shape[0]):
                for j in range(0, mini.shape[1]):
                    if mini.mask[i,j] is True:
                        continue
                    ul = (ma[2] + (j * ma[0]), ma[5] + (i * ma[4]))
                    ur = (ul[0] + ma[0], ul[1])
                    lr = (ur[0], ur[1] + ma[4])
                    ll = (lr[0] - ma[0], lr[1])
                    pixel = Polygon([ul, ur, lr, ll, ul])

                    f_overlap[i,j] = geom.intersection(pixel).area/pixel.area
            mini = np.where(mini < 0, 0, mini)
            average[t] = np.average(mini, weights=f_overlap)

        else:
            average[t] = np.mean(mini)


SM_Daily_Historical=pd.DataFrame()
Data = pd.Series(list((average.values())))
Date = pd.Series(list((average.keys())))
SM_Daily_Historical['Historical SM']=Data
SM_Daily_Historical['Date']=Date
SM_Daily_Historical['Historical SM']=SM_Daily_Historical['Historical SM']/100

In [26]:
Runoff_Daily_Historical['Historical_Runoff_Depth']=Runoff_Daily_Historical['Historical Runoff']*24*60*60/(100**2*1000**2)
Runoff_Daily_Future_245['Future_Runoff_Depth']=Runoff_Daily_Future_245['Future Runoff']*24*60*60/(100**2*1000**2)
Runoff_Daily_Future_585['Future_Runoff_Depth']=Runoff_Daily_Future_585['Future Runoff']*24*60*60/(100**2*1000**2)

In [27]:
CMIP_Historical=pd.merge(LAI_Daily_Historical,Surface_Temp_Daily_Historical,how="left",on="Date")
CMIP_Historical=pd.merge(CMIP_Historical,Runoff_Daily_Historical,how="left",on="Date")
CMIP_Historical=pd.merge(CMIP_Historical,Precip_Daily_Historical,how="left",on="Date")
CMIP_Historical['Historical SM']=SM_Daily_Historical['Historical SM']

In [28]:
CMIP_Future_245=pd.merge(LAI_Daily_Future_245,Surface_Temp_Daily_Future_245,how="left",on="Date")
CMIP_Future_245=pd.merge(CMIP_Future_245,Precip_Daily_Future_245,how="left",on="Date")
CMIP_Future_245=pd.merge(CMIP_Future_245,Runoff_Daily_Future_245,how="left",on="Date")
CMIP_Future_245['Future SM']=SM_Daily_Future_245['Future SM']

In [29]:
CMIP_Future_585=pd.merge(LAI_Daily_Future_585,Surface_Temp_Daily_Future_585,how="left",on="Date")
CMIP_Future_585=pd.merge(CMIP_Future_585,Precip_Daily_Future_585,how="left",on="Date")
CMIP_Future_585=pd.merge(CMIP_Future_585,Runoff_Daily_Future_585,how="left",on="Date")
CMIP_Future_585['Future SM']=SM_Daily_Future_585['Future SM']

In [30]:
CMIP_Future_245['Future Runoff'][CMIP_Future_245['Future Runoff']<0]=0
CMIP_Future_245['Future_Runoff_Depth'][CMIP_Future_245['Future_Runoff_Depth']<0]=0
CMIP_Future_245['Future Precip'][CMIP_Future_245['Future Precip']<0]=0

CMIP_Future_585['Future Runoff'][CMIP_Future_585['Future Runoff']<0]=0
CMIP_Future_585['Future_Runoff_Depth'][CMIP_Future_585['Future_Runoff_Depth']<0]=0
CMIP_Future_585['Future Precip'][CMIP_Future_585['Future Precip']<0]=0

CMIP_Historical['Historical Runoff'][CMIP_Historical['Historical Runoff']<0]=0
CMIP_Historical['Historical_Runoff_Depth'][CMIP_Historical['Historical_Runoff_Depth']<0]=0
CMIP_Historical['Historical Precip'][CMIP_Historical['Historical Precip']<0]=0

In [31]:
CMIP_Historical=pd.merge(LAI_Daily_Historical,Surface_Temp_Daily_Historical,how="left",on="Date")
CMIP_Historical=pd.merge(CMIP_Historical,Runoff_Daily_Historical,how="left",on="Date")
CMIP_Historical=pd.merge(CMIP_Historical,Precip_Daily_Historical,how="left",on="Date")
CMIP_Historical['Historical SM']=SM_Daily_Historical['Historical SM']

CMIP_Historical['Month']=CMIP_Historical['Date']
CMIP_Historical['Day']=CMIP_Historical['Date']
CMIP_Historical['Year']=CMIP_Historical['Date']
for i in range(0,len(CMIP_Historical)):
    CMIP_Historical['Month'][i]=CMIP_Historical['Date'][i].month
    CMIP_Historical['Day'][i]=CMIP_Historical['Date'][i].day
    CMIP_Historical['Year'][i]=CMIP_Historical['Date'][i].year

In [32]:
Future585=pd.read_excel('Path\To\File\CMIP_Future_CMCC-CM2-SR5_ssp585.xlsx')

In [33]:
Future585

,Unnamed: 0,time,Future LAI,Future Surface Temp,Future Precip,Future Runoff,Future_Water_Level,Future SM
0,0,2015-01-01 12:00:00,3.838121,-2.739142,0.489852,44.857170,0.000388,0.355922
1,1,2015-01-02 12:00:00,3.837723,-1.250586,0.118233,41.953804,0.000362,0.354136
2,2,2015-01-03 12:00:00,3.837325,-2.639807,4.808328,74.756454,0.000646,0.350922
3,3,2015-01-04 12:00:00,3.836928,0.213403,58.274538,1842.538622,0.015920,0.349427
4,4,2015-01-05 12:00:00,3.836531,0.795282,6.692663,2015.717619,0.017416,0.350087
...,...,...,...,...,...,...,...,...
31385,31385,2100-12-27 12:00:00,4.242115,5.529382,2.560885,42.546135,0.000368,0.303002
31386,31386,2100-12-28 12:00:00,4.241677,4.985895,0.509547,0.000000,0.000000,0.302482
31387,31387,2100-12-29 12:00:00,4.241239,2.799310,0.435414,0.000000,0.000000,0.296839
31388,31388,2100-12-30 12:00:00,4.240801,2.678125,0.044034,0.000000,0.000000,0.294856


In [34]:
CMIP_Future_585

,Future LAI,Date,Future Surface Temp,Future Precip,Future Runoff,Future_Runoff_Depth,Future SM
0,2.605151,2015-01-01 12:00:00,-0.764327,0.457083,33.519938,0.000290,0.349489
1,2.600231,2015-01-02 12:00:00,0.302564,0.140688,31.185667,0.000269,0.347693
2,2.594458,2015-01-03 12:00:00,-0.748459,3.110215,54.772798,0.000473,0.344723
3,2.587833,2015-01-04 12:00:00,1.362586,53.307013,3571.978031,0.030862,0.343130
4,2.580356,2015-01-05 12:00:00,2.014114,12.791598,2662.400042,0.023003,0.344371
...,...,...,...,...,...,...,...
31385,2.962144,2100-12-27 12:00:00,6.705941,2.549759,90.716366,0.000784,0.300619
31386,2.961739,2100-12-28 12:00:00,6.312170,0.679173,4.444784,0.000038,0.300453
31387,2.961335,2100-12-29 12:00:00,4.447616,0.490163,4.387950,0.000038,0.293890
31388,2.960930,2100-12-30 12:00:00,4.466466,0.084830,2.849799,0.000025,0.292116


In [35]:
Future245=pd.read_excel('Path\To\File\CMIP_Future_CMCC-CM2-SR5_ssp245.xlsx')

In [36]:
Future245

,Unnamed: 0,time,Future LAI,Future Surface Temp,Future Precip,Future Runoff,Future_Water_Level,Future SM
0,0,2015-01-01 12:00:00,3.838119,-2.780066,0.474491,42.640831,0.000368,0.355943
1,1,2015-01-02 12:00:00,3.837719,-1.249274,0.094267,42.512702,0.000367,0.354243
2,2,2015-01-03 12:00:00,3.837319,-3.739783,3.867613,49.620230,0.000429,0.351344
3,3,2015-01-04 12:00:00,3.836919,0.122797,58.027044,1704.995375,0.014731,0.349844
4,4,2015-01-05 12:00:00,3.836519,0.560449,6.198289,1362.164912,0.011769,0.350429
...,...,...,...,...,...,...,...,...
31385,31385,2100-12-27 12:00:00,5.458473,3.513849,1.380380,46.185296,0.000399,0.313427
31386,31386,2100-12-28 12:00:00,5.457922,3.503595,0.188061,20.708437,0.000179,0.308367
31387,31387,2100-12-29 12:00:00,5.457371,2.591272,0.331161,21.594919,0.000187,0.304838
31388,31388,2100-12-30 12:00:00,5.456821,5.868982,5.823245,156.621427,0.001353,0.315203


In [37]:
CMIP_Future_245

,Future LAI,Date,Future Surface Temp,Future Precip,Future Runoff,Future_Runoff_Depth,Future SM
0,2.605149,2015-01-01 12:00:00,-0.791463,0.430536,32.398227,0.000280,0.349545
1,2.600224,2015-01-02 12:00:00,0.214189,0.101325,31.547198,0.000273,0.347820
2,2.594446,2015-01-03 12:00:00,-1.584178,2.493361,36.803930,0.000318,0.345156
3,2.587816,2015-01-04 12:00:00,1.290696,53.556986,3485.911470,0.030118,0.343541
4,2.580334,2015-01-05 12:00:00,1.833790,12.799080,2203.839470,0.019041,0.344263
...,...,...,...,...,...,...,...
31385,3.576860,2100-12-27 12:00:00,4.816925,1.117277,34.570924,0.000299,0.311097
31386,3.576411,2100-12-28 12:00:00,4.777284,0.164436,16.270391,0.000141,0.305668
31387,3.575962,2100-12-29 12:00:00,4.138953,0.372256,17.917037,0.000155,0.302607
31388,3.575514,2100-12-30 12:00:00,6.578263,4.656198,211.694466,0.001829,0.310960


In [38]:
CMIP_Future_245.to_excel('Path\To\File\CMIP_Future_CMCC-CM2-SR5_ssp245_Polygon.xlsx')
CMIP_Future_585.to_excel('Path\To\File\CMIP_Future_CMCC-CM2-SR5_ssp585_Polygon.xlsx')
CMIP_Historical.to_excel('Path\To\File\CMIP_Historical_CMCC-CM2-SR5_Polygon.xlsx')